In [33]:
!pip install vaderSentiment


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [34]:
import requests
import pandas as pd
import numpy as np
from datetime import date, timedelta
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import csv

In [46]:
NEWS_ENDPOINT = "https://newsapi.org/v2/everything?"

news_api_key = "1b3b397afaa34a7b8f1589a45af74273"

current_date =date.today()
days_to_subtract = 31
last_30_days = current_date - timedelta(days=days_to_subtract)

news_params = {
    'apiKey': "1b3b397afaa34a7b8f1589a45af74273",
    'qInTitle': "nvidia",
    'from': last_30_days,
    'to': current_date,
    'language': 'en',
    'sortBy': 'relevancy'
}

In [47]:
# API URL (Replace with your actual endpoint)
api_url = "https://newsapi.org/v2/everything?"

# Optional: Include API headers or parameters if required
headers = {
    "apiKey": "1b3b397afaa34a7b8f1589a45af74273",  # If required
    # "Content-Type": "application/json",
    'qInTitle': "nvidia",
    'from': last_30_days,
    'to': current_date,
    'language': 'en',
    'sortBy': 'relevancy'
}

# Fetch data from the API
response = requests.get(api_url, params=headers)

# Check if the request was successful
if response.status_code == 200:
    news_articles = response.json()['articles']  # Convert response to JSON
    print(news_articles)

    # # Define CSV filename
    csv_filename = "news_data.csv"

    # # Define column headers
    headers = ["Headline", "Brief", "URL", "Date"]

    # Extract relevant data
    formatted_data = []
    for article in news_articles:  
        formatted_data.append({
            "Headline": article.get("title", ""),  # Adjust key names if different
            "Brief": article.get("description", ""),
            "URL": article.get("url", ""),
            "Date": article.get("publishedAt", "")
        })

    # Write to CSV
    with open(csv_filename, mode="w", newline="", encoding="utf-8") as file:
        writer = csv.DictWriter(file, fieldnames=headers)
        writer.writeheader()  # Write column headers
        writer.writerows(formatted_data)  # Write extracted data

    print(f"CSV file '{csv_filename}' has been created successfully with {len(formatted_data)} articles!")

else:
    print(f"Error fetching data: {response.status_code}, {response.text}")

Error fetching data: 426, {"status":"error","code":"parameterInvalid","message":"You are trying to request results too far in the past. Your plan permits you to request articles as far back as 2024-12-29, but you have requested 2024-12-26. You may need to upgrade to a paid plan."}


In [37]:
# Load the existing CSV file
csv_filename = "news_data.csv"
df = pd.read_csv(csv_filename)

# Initialize VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

# Function to classify sentiment
def get_sentiment(text):
    score = analyzer.polarity_scores(text)["compound"]
    if score > 0.05:
        return "Positive"
    elif score < -0.05:
        return "Negative"
    else:
        return "Neutral"

# Apply sentiment analysis to each row
df["Sentiment"] = df.apply(lambda row: get_sentiment(f"{row['Headline']} {row['Brief']}"), axis=1)

# Save back to the same CSV file (overwrite with sentiment added)
df.to_csv(csv_filename, index=False)

print(f"Sentiment appended to {csv_filename} successfully!")

Sentiment appended to news_data.csv successfully!
